## Train GPT on addition

Train a GPT model on a dedicated addition dataset to see if a Transformer can learn to add.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import jax
import jax.numpy as jnp
import haiku as hk

In [4]:
jax.default_backend()

09/27/2021 08:39:24 - INFO - absl -   Starting the local TPU driver.
09/27/2021 08:39:24 - INFO - absl -   Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: local://
09/27/2021 08:39:24 - INFO - absl -   Unable to initialize backend 'tpu': tpu_client_timer_callback() got an unexpected keyword argument 'timer'


'gpu'

In [5]:
from tqdm import tqdm
import math
import numpy as np
from typing import Mapping
import functools
from functools import partial

import torch
from torch.utils.data import Dataset

In [6]:
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    """
    Returns addition problems of up to some number of digits in the inputs. Recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
    encoding will simply be the n-digit first number, n-digit second number, 
    and (n+1)-digit result, all simply concatenated together. Because each addition
    problem is so structured, there is no need to bother the model with encoding
    +, =, or other tokens. Each possible sequence has the same length, and simply
    contains the raw digits of the addition problem.
    
    As a few examples, the 2-digit problems:
    - 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
    - 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
    etc.
    
    We will also only train GPT on the final (n+1)-digits because the first
    two n-digits are always assumed to be given. So when we give GPT an exam later,
    we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
    to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5]
    in 3 sequential steps.
    
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        idx = self.ixes[idx]
        nd = 10**self.ndigit
        a = idx // nd
        b = idx %  nd
        c = a + b
        render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028" 
        dix = [int(s) for s in render] # convert each character to its token index
        # x will be input to GPT and y will be the associated expected outputs
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long) # predict the next token in the sequence
        y[:self.ndigit*2-1] = -100 # we will only train in the output locations. -100 will mask loss to zero
        return x, y


In [7]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset = AdditionDataset(ndigit=ndigit, split='train')
test_dataset = AdditionDataset(ndigit=ndigit, split='test')

In [8]:
train_dataset[0] # sample a training instance just to see what one raw example looks like

(tensor([4, 7, 1, 7, 0, 6]), tensor([-100, -100, -100,    0,    6,    4]))

In [9]:
from mingpt.model import gpt, loss_fn, GPTConfig

rng = jax.random.PRNGKey(42)
# initialize a baby GPT model
gpt_config = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, 
                  n_layer=2, n_head=4, n_embd=128)
hk_loss_fn = hk.transform(partial(loss_fn, config=gpt_config, is_training=True))

In [10]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
rng, subkey = jax.random.split(rng)
tconf = TrainerConfig(max_epochs=50, batch_size=512//2, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset)*(ndigit+1),
                      num_workers=4, rng=subkey, step_tokens=3)
trainer = Trainer(hk_loss_fn, train_dataset, test_dataset, tconf)

In [11]:
params = trainer.init_params() 

09/27/2021 08:39:31 - INFO - mingpt.trainer -   number of parameters: 384384


In [12]:
params, _ = trainer.train(params)

epoch 1 iter 35: train loss 1.53915. lr 5.994563e-04: 100%|██████████| 36/36 [00:20<00:00,  1.74it/s]
09/27/2021 08:39:55 - INFO - mingpt.trainer -   test loss: 1.5366647243499756
epoch 2 iter 71: train loss 1.15609. lr 5.976741e-04: 100%|██████████| 36/36 [00:00<00:00, 37.42it/s]
09/27/2021 08:39:56 - INFO - mingpt.trainer -   test loss: 1.2317906618118286
epoch 3 iter 107: train loss 1.03235. lr 5.946582e-04: 100%|██████████| 36/36 [00:00<00:00, 37.02it/s]
09/27/2021 08:39:58 - INFO - mingpt.trainer -   test loss: 0.9656183123588562
epoch 4 iter 143: train loss 0.49207. lr 5.904211e-04: 100%|██████████| 36/36 [00:01<00:00, 35.54it/s]
09/27/2021 08:40:00 - INFO - mingpt.trainer -   test loss: 0.514510452747345
epoch 5 iter 179: train loss 0.29771. lr 5.849804e-04: 100%|██████████| 36/36 [00:00<00:00, 36.86it/s]
09/27/2021 08:40:01 - INFO - mingpt.trainer -   test loss: 0.41415607929229736
epoch 6 iter 215: train loss 0.33071. lr 5.783586e-04: 100%|██████████| 36/36 [00:00<00:00, 37.24

epoch 46 iter 1655: train loss 0.00053. lr 6.000000e-05: 100%|██████████| 36/36 [00:00<00:00, 37.05it/s]
09/27/2021 08:41:05 - INFO - mingpt.trainer -   test loss: 0.008746650069952011
epoch 47 iter 1691: train loss 0.00759. lr 6.000000e-05: 100%|██████████| 36/36 [00:00<00:00, 37.25it/s]
09/27/2021 08:41:07 - INFO - mingpt.trainer -   test loss: 0.014044404961168766
epoch 48 iter 1727: train loss 0.00966. lr 6.000000e-05: 100%|██████████| 36/36 [00:00<00:00, 36.96it/s]
09/27/2021 08:41:08 - INFO - mingpt.trainer -   test loss: 0.01291092298924923
epoch 49 iter 1763: train loss 0.00773. lr 6.000000e-05: 100%|██████████| 36/36 [00:00<00:00, 37.80it/s]
09/27/2021 08:41:10 - INFO - mingpt.trainer -   test loss: 0.014307268895208836
epoch 50 iter 1799: train loss 0.00182. lr 6.000000e-05: 100%|██████████| 36/36 [00:00<00:00, 36.08it/s]
09/27/2021 08:41:11 - INFO - mingpt.trainer -   test loss: 0.011349497362971306


In [13]:
# now let's give the trained model an addition exam
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import sample

model = hk.transform(partial(gpt, config=gpt_config, is_training=False))
model = hk.without_apply_rng(model).apply

def give_exam(dataset, batch_size=32, max_batches=-1):
    batch_size=1024
    max_batches=10
    results = []
    loader = DataLoader(dataset, batch_size=batch_size)
    for b, (x, y) in enumerate(loader):
        d1d2 = jnp.array(x[:, :ndigit*2])
        batched_sample = partial(sample, params=params, model=model, config=gpt_config, steps=ndigit+1)
        d1d2d3 = jax.vmap(batched_sample)(x=jnp.array(d1d2))
        #d1d2d3 = sample(params, model, gpt_config, d1d2, ndigit+1
        d3 = d1d2d3[:, -(ndigit+1):]
        factors = jnp.array([[10**i for i in range(ndigit+1)][::-1]])
        # decode the integers from individual digits
        d1i = (d1d2[:,:ndigit] * factors[:,1:]).sum(1)
        d2i = (d1d2[:,ndigit:ndigit*2] * factors[:,1:]).sum(1)
        d3i_pred = (d3 * factors).sum(1)
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt) # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))
        
        if max_batches >= 0 and b+1 >= max_batches:
            break
    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [14]:
# training set: how well did we memorize?
give_exam(train_dataset, batch_size=1024, max_batches=10)

GPT claims that 053 + 047 = 090 (gt is 100; NOPE)
GPT claims that 058 + 042 = 090 (gt is 100; NOPE)
GPT claims that 057 + 043 = 090 (gt is 100; NOPE)
final score: 8997/9000 = 99.97% correct


In [15]:
# test set: how well did we generalize?
give_exam(test_dataset, batch_size=1024, max_batches=-1)

GPT claims that 031 + 069 = 090 (gt is 100; NOPE)
final score: 999/1000 = 99.90% correct


In [16]:
# well that's amusing... we're lucky, sometimes the model can't generalize a couple of test cases